In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
from pathlib import Path

from gensim.utils import deaccent
import ftfy
import textdistance

import shapefile
import shapely
from shapely.geometry import Point

pd.set_option('max_columns', None)

In [3]:
df = pd.read_csv('../Data/Monash/RDI_Africa_ADM2_1990_2015.csv', encoding='latin')
df.rename(columns={'ADM1': 'adm1_name', 'ADM2': 'adm2_name'}, inplace=True)
print(df.shape)
df.head()

(155168, 73)


,ADM2ID,countrycode,countryname,adm1_name,adm2_name,year,ADM2_lat,ADM2_lon,frac_LU_artificial,frac_LU_cropland,frac_LU_grass,frac_LU_tree,frac_LU_shrubs,frac_LU_herbaceous,frac_LU_mangroves,frac_LU_sparse,frac_LU_baresoil,frac_LU_snow,frac_LU_water,mean_light,sum_light,UCDP_death,UCDP_event,battle,headquarters,nonviolent,remoteviol,riot,strategic_dev,violence_civ,ACLED_death,mines,Aluminum_mine,Asbestos_mine,BariumBarite_mine,Beryllium_mine,BoronBorates_mine,Chromium_mine,Copper_mine,Diamond_mine,Gold_mine,Iron_mine,Lead_mine,Manganese_mine,Mica_mine,Nickel_mine,PhosphorusPhosphates_mine,Platinum_mine,REE_mine,Silver_mine,Sulfur_mine,Strontium_mine,Tin_mine,Tungsten_mine,Uranium_mine,Zinc_mine,ports,port_oil_terminal,HighwayKM,PrimaryKM,SecondaryKM,TertiaryKM,Local_UrbanKM,TrailKM,PrivateKM,UnspecifiedKM,total_road_length,oil_field,gas_field,Power_Plants,Power_Plant_Hydro,Power_Plant_Thermal,Power_Plant_OTHER
0,12036,DZA,Algeria,Adrar,Adrar,1990,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN
1,12036,DZA,Algeria,Adrar,Adrar,1991,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN
2,12036,DZA,Algeria,Adrar,Adrar,1992,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,1.80942,1652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN
3,12036,DZA,Algeria,Adrar,Adrar,1993,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,1.88499,1721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN
4,12036,DZA,Algeria,Adrar,Adrar,1994,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,1.74261,1591.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN


In [2]:
shps = glob('../Shapefiles/preprocessed/*.shp')
shapes_by_country = {}
for i, shp in tqdm(enumerate(shps), total=len(shps)):
    codes = Path(shp).name.replace('.shp', '').split('_')
    shp = shapefile.Reader(shp)
    shapes = [shapely.geometry.shape(s) for s in shp.shapes()]
    records = shp.records()
    
    for code in codes:
        shapes_by_country[code] = []
        for shape, record in zip(shapes, records):
            shapes_by_country[code].append((shape, *record))
    
assert 'GT' in shapes_by_country and 'PM' in shapes_by_country

100%|██████████| 276/276 [02:31<00:00,  1.83it/s]


In [4]:
def country_shapes(country):
    if country in shapes_by_country:
        print('Found country', country)
        yield from shapes_by_country[country]
    else:
        for shapes in shapes_by_country.values():
            yield from shapes


coordinates = [(14.24767, -16.417101), (13.78871, -16.93281)]
for coor in coordinates:
    point = Point(coor)
    for shape, adm0, adm1, adm2 in country_shapes('AGO'):
            if point.within(shape):
                print(coor, 'in', adm2)

Found country AGO
(14.24767, -16.417101) in AGO.8.1_1
Found country AGO
(13.78871, -16.93281) in AGO.8.1_1


In [5]:
locs = df[['ADM2_lon', 'ADM2_lat', 'countrycode']].drop_duplicates()
locs['adm0'] = np.nan
locs['adm1'] = np.nan
locs['adm2'] = np.nan
locs['nearest_loc'] = False

In [6]:
def country_shapes(country):
    if country in shapes_by_country:
        yield from shapes_by_country[country]
    else:
        for shapes in shapes_by_country.values():
            yield from shapes
            

pbar = tqdm(total=locs.shape[0], desc='Finding codes')
for idx, row in locs.iterrows():
    if isinstance(row['adm1'], str):
        pbar.update(1)
        continue
    
    coordinates = (row['ADM2_lon'], row['ADM2_lat'])
    point = Point(coordinates)
    nearest_record = None
    nearest_distance = np.inf
    for shape, adm0, adm1, adm2 in country_shapes(row['countrycode']):
        if point.within(shape):
            locs.loc[idx, 'adm2'] = adm2
            locs.loc[idx, 'adm1'] = adm1
            locs.loc[idx, 'adm0'] = adm0
            break
            
        distance = point.distance(shape)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_record = (adm0, adm1, adm2)
    else:
        if nearest_record:
            locs.loc[idx, 'nearest_loc'] = True
            locs.loc[idx, 'adm2'] = nearest_record[2]
            locs.loc[idx, 'adm1'] = nearest_record[1]
            locs.loc[idx, 'adm0'] = nearest_record[0]
        
    pbar.update(1)
pbar.close()
print('Total nearest:', locs['nearest_loc'].sum())
print('Missing:', locs.loc[locs['adm1'].isnull(), 'countrycode'].unique())

Finding codes: 100%|██████████| 5842/5842 [02:21<00:00, 41.33it/s]  

Total nearest: 26
Missing: []


In [5]:
df = df.merge(locs, on=['ADM2_lon', 'ADM2_lat', 'countrycode'], how='left')

for adm in ['adm1', 'adm2']:
    mask = np.logical_or(df[f'{adm}_name'].isnull(), df[f'{adm}_name'] == '')
    df.loc[mask, adm] = np.nan

In [6]:
df.to_csv('../Output/Monash/Monash.csv', index=False)

# String match

In [2]:
def clean_text(s):
    if not isinstance(s, str):
        return s
    
    s = deaccent(ftfy.fix_text(s))
    try:
        s = s.strip().upper().encode('cp1252').decode().lower()
        s = deaccent(s)
    except Exception as e:
        s = s.lower()
        
    return s


def find_closest_match(codes, adm0, adm1, adm2, max_allowed_distance=3):
    sub_codes = codes[codes['adm0'] == adm0].copy()
    sub2_codes = sub_codes[np.logical_and(sub_codes['adm1_name'] == adm1, 
                                          sub_codes['adm2'] != 'NA')].copy()
    if sub2_codes.shape[0] > 0:
        sub_codes = sub2_codes.copy()
        s = adm2
        col = 'adm2_name'
    else:
        s = adm1
        col = 'adm1_name'
        
    sub_codes['fuzzy_match_on'] = col
    
    sub_codes['distance'] = sub_codes[col].apply(lambda x: textdistance.damerau_levenshtein(x, s))
    min_distance = sub_codes['distance'].min()
    
    if min_distance > max_allowed_distance:
        return None

    sub_codes = sub_codes[sub_codes['distance'] == min_distance]
    
    if sub_codes.shape[0] == 1:
        return sub_codes.iloc[0]
    elif sub_codes.shape[0] == 0:
        return None
    else:
        sub_codes['distance'] = sub_codes[col].apply(lambda x: textdistance.mra(x, s))
        return sub_codes.iloc[sub_codes['distance'].argmin()]
        
    return sub_codes

In [3]:
df = pd.read_csv('../Output/Monash/Monash.csv')

for adm in ['adm1', 'adm2']:
    df[adm + '_clean'] = df[adm + '_name'].apply(clean_text)

df.head()

,ADM2ID,countrycode,countryname,adm1_name,adm2_name,year,ADM2_lat,ADM2_lon,frac_LU_artificial,frac_LU_cropland,frac_LU_grass,frac_LU_tree,frac_LU_shrubs,frac_LU_herbaceous,frac_LU_mangroves,frac_LU_sparse,frac_LU_baresoil,frac_LU_snow,frac_LU_water,mean_light,sum_light,UCDP_death,UCDP_event,battle,headquarters,nonviolent,remoteviol,riot,strategic_dev,violence_civ,ACLED_death,mines,Aluminum_mine,Asbestos_mine,BariumBarite_mine,Beryllium_mine,BoronBorates_mine,Chromium_mine,Copper_mine,Diamond_mine,Gold_mine,Iron_mine,Lead_mine,Manganese_mine,Mica_mine,Nickel_mine,PhosphorusPhosphates_mine,Platinum_mine,REE_mine,Silver_mine,Sulfur_mine,Strontium_mine,Tin_mine,Tungsten_mine,Uranium_mine,Zinc_mine,ports,port_oil_terminal,HighwayKM,PrimaryKM,SecondaryKM,TertiaryKM,Local_UrbanKM,TrailKM,PrivateKM,UnspecifiedKM,total_road_length,oil_field,gas_field,Power_Plants,Power_Plant_Hydro,Power_Plant_Thermal,Power_Plant_OTHER,adm0,adm1,adm2,nearest_loc,adm1_clean,adm2_clean
0,12036,DZA,Algeria,Adrar,Adrar,1990,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN,DZA,DZA.1_1,DZA.1.1_1,False,adrar,adrar
1,12036,DZA,Algeria,Adrar,Adrar,1991,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN,DZA,DZA.1_1,DZA.1.1_1,False,adrar,adrar
2,12036,DZA,Algeria,Adrar,Adrar,1992,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,1.80942,1652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN,DZA,DZA.1_1,DZA.1.1_1,False,adrar,adrar
3,12036,DZA,Algeria,Adrar,Adrar,1993,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,1.88499,1721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN,DZA,DZA.1_1,DZA.1.1_1,False,adrar,adrar
4,12036,DZA,Algeria,Adrar,Adrar,1994,27.967939,-0.1794,0.0,0.000703,0.0,0.000176,0.000176,0.0,0.0,0.001889,0.997057,0.0,0.0,1.74261,1591.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.714579,0.0,33.798156,36.512736,0.0,1.0,NaN,NaN,NaN,NaN,DZA,DZA.1_1,DZA.1.1_1,False,adrar,adrar


In [4]:
codes = pd.read_csv('../Data/GADM_codes.csv')
codes.fillna('NA', inplace=True)

for col in ['adm1_name', 'adm2_name']:
    codes[col] = codes[col].apply(clean_text)
    
codes.columns = [x.lower() for x in codes.columns]
codes.head()

,adm0,adm0_name,adm1,adm1_name,adm2,adm2_name
0,ABW,Aruba,NA,na,NA,na
1,AFG,Afghanistan,NA,na,NA,na
2,AGO,Angola,NA,na,NA,na
3,AIA,Anguilla,NA,na,NA,na
4,ALA,Åland,NA,na,NA,na


In [5]:
df = df.merge(codes, 
             left_on=['countrycode', 'adm1_clean', 'adm2_clean'],
             right_on=['adm0', 'adm1_name', 'adm2_name'], 
             how='left',
             suffixes=['', '_y'])

mask = df['adm0_y'].notnull()
df['string_match'] = np.where(mask, True, False)
print(mask.sum(), mask.mean())

locs_equal_string = df.loc[df['adm2_y'].notnull(), 'adm2'] == df.loc[df['adm2_y'].notnull(), 'adm2_y']
print(locs_equal_string.sum(), locs_equal_string.mean())

cols = df.filter(regex=r'.*_y$').columns
for col in cols:
    df.loc[mask, col[:-2]] = df.loc[mask, col]
    
df.drop(columns=list(cols) + ['adm1_clean', 'adm2_clean', 'adm0_name'], inplace=True)

df.sample(10)

102440 0.6601876675603218
98228 0.9588832487309644


,ADM2ID,countrycode,countryname,adm1_name,adm2_name,year,ADM2_lat,ADM2_lon,frac_LU_artificial,frac_LU_cropland,frac_LU_grass,frac_LU_tree,frac_LU_shrubs,frac_LU_herbaceous,frac_LU_mangroves,frac_LU_sparse,frac_LU_baresoil,frac_LU_snow,frac_LU_water,mean_light,sum_light,UCDP_death,UCDP_event,battle,headquarters,nonviolent,remoteviol,riot,strategic_dev,violence_civ,ACLED_death,mines,Aluminum_mine,Asbestos_mine,BariumBarite_mine,Beryllium_mine,BoronBorates_mine,Chromium_mine,Copper_mine,Diamond_mine,Gold_mine,Iron_mine,Lead_mine,Manganese_mine,Mica_mine,Nickel_mine,PhosphorusPhosphates_mine,Platinum_mine,REE_mine,Silver_mine,Sulfur_mine,Strontium_mine,Tin_mine,Tungsten_mine,Uranium_mine,Zinc_mine,ports,port_oil_terminal,HighwayKM,PrimaryKM,SecondaryKM,TertiaryKM,Local_UrbanKM,TrailKM,PrivateKM,UnspecifiedKM,total_road_length,oil_field,gas_field,Power_Plants,Power_Plant_Hydro,Power_Plant_Thermal,Power_Plant_OTHER,adm0,adm1,adm2,nearest_loc,string_match
44021,3192,BEN,Benin,borgou,nikki,1993,9.975020,3.206300,0.000000,0.332044,0.025972,0.127967,0.514017,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011586,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,83.551614,83.551614,0.0,0.0,NaN,NaN,NaN,NaN,BEN,BEN.4_1,BEN.4.4_1,False,True
44722,3219,BEN,Benin,OuÚmÚ,Adjarra,1992,6.470690,2.683750,0.005601,0.218022,0.542212,0.109134,0.082366,0.000000,0.016144,0.000000,0.000000,0.0,0.026522,4.289470,326.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,BEN,BEN.10_1,BEN.10.1_1,False,False
78911,51655,LBY,Libya,Ghadamis,NaN,1991,29.885500,10.740500,0.000252,0.000263,0.000000,0.000000,0.015412,0.000015,0.000000,0.027534,0.956452,0.0,0.000072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,172.512836,0.0,4795.127870,4967.640706,1.0,1.0,NaN,NaN,NaN,NaN,LBY,LBY.17_1,NaN,False,False
134031,38970,ZAF,South Africa,Orange Free State,Reitz,1991,-27.784790,28.477160,0.003173,0.996812,0.000000,0.000015,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.912242,386.385961,302.032198,0.0,0.000000,0.0,0.000000,809.330402,0.0,0.0,NaN,NaN,NaN,NaN,ZAF,ZAF.2_1,ZAF.2.4_1,False,False
5557,12317,DZA,Algeria,biskra,m_ziraa,2009,34.876438,6.303920,0.000000,0.119292,0.000354,0.032250,0.036627,0.000000,0.000000,0.517208,0.294268,0.0,0.000000,0.082232,112.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,23.727606,23.727606,0.0,0.0,NaN,NaN,NaN,NaN,DZA,DZA.9_1,DZA.9.23_1,False,True
6811,12366,DZA,Algeria,Bordj Bou ArrÚridj,El Anseur,2015,36.037590,4.842010,0.002052,0.295434,0.321293,0.123243,0.110621,0.000000,0.000000,0.146229,0.001129,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,DZA,DZA.11_1,DZA.11.13_1,False,False
13764,12565,DZA,Algeria,el bayadh,sidi ameur,2000,33.567669,1.442270,0.000000,0.000043,0.000000,0.002062,0.002025,0.000000,0.000000,0.308263,0.687607,0.0,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,DZA,DZA.17_1,DZA.17.18_1,False,True
1

In [6]:
df.to_csv('../Output/Monash/Monash.csv', index=False)